In [2]:
# Ensure steel_selector.py is in the same directory or accessible in PYTHONPATH
from steel_selector import SteelSelector
import ipywidgets as widgets
from IPython.display import display, clear_output
from indeterminatebeam import Beam, Support, PointLoadV, DistributedLoadV, PointTorque
import traceback # For detailed error messages

# --- Steel Selector Setup ---
# IMPORTANT: Verify units in your CSV and adjust SteelSelector's scaling if needed.
# The goal is for selector1.get_properties()["Ix"] to be in mm^4.
selector1 = SteelSelector(display_geom_props=["Ix", "Ag", "Sx", "d"]) # Add other props if needed
selector1.launch()

# --- Beam and Load Definition UI ---
wide_layout = widgets.Layout(width='auto', min_width='250px') # Allow more flexible width
input_layout = widgets.Layout(width='150px') # For numeric inputs

beam_span_widget = widgets.FloatText(value=6.0, description="Span (m):", layout=input_layout, style={'description_width': 'initial'})
support_count = widgets.BoundedIntText(value=2, min=1, max=10, description="Num Supports:", layout=input_layout, style={'description_width': 'initial'})
support_inputs_box = widgets.VBox([])
load_count = widgets.BoundedIntText(value=1, min=0, max=10, description="Num Loads:", layout=input_layout, style={'description_width': 'initial'})
load_inputs_box = widgets.VBox([])

support_type_options = {"Roller": (0, 1, 0), "Pinned": (1, 1, 0), "Fixed": (1, 1, 1)}

def update_support_inputs(change=None):
    widgets_list = []
    L = beam_span_widget.value
    for i in range(support_count.value):
        default_loc = 0.0 if i == 0 else L if i == 1 else L / 2
        loc = widgets.FloatText(value=default_loc, description=f"S{i+1} Loc (m):", layout=input_layout, style={'description_width': 'initial'})
        
        default_type = "Pinned" if i == 0 else "Roller" if i == 1 else "Pinned"
        typ_dropdown = widgets.Dropdown(options=list(support_type_options.keys()), value=default_type, description=f"S{i+1} Type:", layout=wide_layout, style={'description_width': 'initial'})
        widgets_list.append(widgets.HBox([loc, typ_dropdown]))
    support_inputs_box.children = widgets_list

from functools import partial
def update_load_inputs(change=None):
    widgets_list = []
    L = beam_span_widget.value
    for i in range(load_count.value):
        typ_dropdown = widgets.Dropdown(options=["Point Load", "UDL", "Torque"], description=f"L{i+1} Type:", layout=wide_layout, style={'description_width': 'initial'})
        mag = widgets.FloatText(value=-10.0, description="Mag (kN/kNm):", layout=input_layout, style={'description_width': 'initial'}) # Negative for downward
        x1_default = L / (load_count.value + 1) * (i + 1) # Distribute loads somewhat
        x1 = widgets.FloatText(value=x1_default, description="x1/Pos (m):", layout=input_layout, style={'description_width': 'initial'})
        x2 = widgets.FloatText(value=x1_default + L/4 if x1_default + L/4 <=L else L, description="x2 (m):", layout=input_layout, style={'description_width': 'initial'})
        x2_box = widgets.VBox([x2])

        def toggle_x2_visibility(change, box_to_toggle):
            box_to_toggle.layout.display = 'flex' if change['new'] == "UDL" else 'none'
        
        initial_display = 'flex' if typ_dropdown.value == "UDL" else 'none'
        x2_box.layout.display = initial_display
        typ_dropdown.observe(partial(toggle_x2_visibility, box_to_toggle=x2_box), names='value')
        widgets_list.append(widgets.HBox([typ_dropdown, mag, x1, x2_box]))
    load_inputs_box.children = widgets_list

beam_span_widget.observe(lambda c: (update_support_inputs(), update_load_inputs()), names="value")
support_count.observe(update_support_inputs, names="value")
load_count.observe(update_load_inputs, names="value")

update_support_inputs()
update_load_inputs()

# --- UI Display ---
ui_box = widgets.VBox([
    widgets.HTML("<h3>🔗 Steel Section (selected via UI above)</h3>"),
    widgets.HTML("<h3>📏 Beam Definition</h3>"),
    beam_span_widget,
    support_count, support_inputs_box,
    widgets.HTML("<h3>➕ Loads</h3>"),
    load_count, load_inputs_box
])
display(ui_box)

# --- Analysis and Plotting ---
output_area = widgets.Output() # To capture prints and plots separately

# Global variable to store the latest analyzed beam for debugging or re-plotting if needed.
# Try to avoid globals, but can be useful in notebooks.
latest_analyzed_beam = None

def run_analysis_and_plot_action(button_event):
    global latest_analyzed_beam
    
    with output_area: # All prints and plots will go into this output widget
        clear_output(wait=True) # Clear previous output in this area
        print("Starting analysis...\n")

        # 1. Get Steel Properties (assuming Ix from selector is in mm^4)
        selected_props = selector1.get_properties() # Get all available scaled props
        Ix_mm4 = selected_props.get("Ix")

        if Ix_mm4 is None or Ix_mm4 <= 0:
            print(f"❌ Error: Invalid Ix ({Ix_mm4}). Please select a valid steel section with Ix > 0.")
            print(f"  Selected properties from SteelSelector: {selected_props}")
            return

        E_mpa = 200000  # Modulus of Elasticity for steel in MPa (N/mm^2)
        E_pa = E_mpa * 1e6  # Convert MPa to Pa (N/m^2)
        I_m4 = Ix_mm4 * 1e-12  # Convert mm^4 to m^4

        print(f"--- Section Properties ---")
        print(f"  Ix from selector: {Ix_mm4:.3e} mm⁴")
        print(f"  Using E: {E_pa / 1e9:.1f} GPa")
        print(f"  Using I: {I_m4:.3e} m⁴")
        if I_m4 <= 1e-12: # Arbitrary small number check
             print(f"⚠️ Warning: Moment of Inertia I ({I_m4:.2e} m^4) is very small. Check CSV units and SteelSelector scaling.")


        # 2. Create Beam
        L_m = beam_span_widget.value
        if L_m <= 0:
            print("❌ Error: Beam span must be positive.")
            return
        
        print(f"\n--- Beam Definition ---")
        print(f"  Length (L): {L_m} m")
        
        # Use a specific E and I for the beam object
        # The indeterminatebeam library may have default E, I if not provided.
        # It's better to be explicit.
        current_beam = Beam(L_m, E=E_pa, I=I_m4)

        # 3. Add Supports
        print("\n--- Supports ---")
        if not support_inputs_box.children:
            print("⚠️ No support widgets found. Ensure 'Num Supports' > 0.")
        num_supports_added_to_beam = 0
        for i, row_hbox in enumerate(support_inputs_box.children):
            try:
                x_loc = row_hbox.children[0].value
                s_type_name = row_hbox.children[1].value
                dof = support_type_options[s_type_name]
                current_beam.add_supports(Support(x_loc, dof))
                print(f"  Added: {s_type_name} at {x_loc:.2f} m")
                num_supports_added_to_beam +=1
            except Exception as e:
                print(f"❌ Error adding support S{i+1}: {e}")
                traceback.print_exc()
                return 
        if num_supports_added_to_beam == 0 and support_count.value > 0:
            print("❌ Error: Supports were defined in UI, but none were added to beam object. Check for errors above.")
            return
        elif support_count.value == 0:
            print("⚠️ No supports defined by user.")


        # 4. Add Loads
        print("\n--- Loads ---")
        if not load_inputs_box.children and load_count.value > 0 :
             print("⚠️ Load count > 0 but no load widgets found.")
        num_loads_added_to_beam = 0
        for i, row_hbox in enumerate(load_inputs_box.children):
            try:
                l_type = row_hbox.children[0].value
                mag_input = row_hbox.children[1].value # kN or kNm
                mag_N_or_Nm = mag_input * 1000    # Convert to N or Nm
                x1_m = row_hbox.children[2].value
                
                load_str = f"  Adding L{i+1}: {l_type}, Mag={mag_N_or_Nm:.1f} N (or Nm), @x1={x1_m:.2f}m"
                
                if l_type == "Point Load":
                    current_beam.add_loads(PointLoadV(mag_N_or_Nm, x1_m))
                elif l_type == "UDL":
                    x2_m = row_hbox.children[3].children[0].value # x2 is inside VBox
                    if x1_m >= x2_m:
                        print(f"❌ Error for Load L{i+1} (UDL): x1 ({x1_m}) must be less than x2 ({x2_m}). Skipping this load.")
                        continue
                    current_beam.add_loads(DistributedLoadV(mag_N_or_Nm, (x1_m, x2_m)))
                    load_str += f", x2={x2_m:.2f}m"
                elif l_type == "Torque":
                    current_beam.add_loads(PointTorque(mag_N_or_Nm, x1_m))
                print(load_str)
                num_loads_added_to_beam +=1
            except Exception as e:
                print(f"❌ Error adding load L{i+1}: {e}")
                traceback.print_exc()
                # Decide whether to return or just skip the load
        if num_loads_added_to_beam == 0 and load_count.value > 0:
            print("⚠️ Loads were defined in UI, but none were added to beam object. Check for errors above.")
        elif load_count.value == 0:
            print("  No loads defined by user.")


        # 5. Analyse
        print("\n--- Analysis ---")
        try:
            current_beam.analyse()
            print("✅ Beam analysis successful.")
            latest_analyzed_beam = current_beam # Store for potential later use
        except Exception as e:
            print(f"❌ Beam analysis failed: {e}")
            print("  Beam object details before crash:")
            print(f"    Length: {current_beam.length}, E: {current_beam.E}, I: {current_beam.I}")
            print(f"    Supports: {current_beam.supports}")
            print(f"    Loads: {current_beam.loads}")
            traceback.print_exc()
            latest_analyzed_beam = current_beam # Store partially processed beam
            return

        # 6. Plot Results
        print("\n--- Plotting ---")
        try:
            print("  Generating external forces plot...")
            fig_ext = current_beam.plot_beam_external()
            if fig_ext: # Check if a figure object was returned
                fig_ext.show() # Use .show() for matplotlib figures
                print("  External forces plot displayed.")
            else:
                print("  ⚠️ plot_beam_external() did not return a figure.")

            print("  Generating internal forces plot...")
            fig_int = current_beam.plot_beam_internal()
            if fig_int: # Check if a figure object was returned
                fig_int.show()
                print("  Internal forces plot displayed.")
            else:
                print("  ⚠️ plot_beam_internal() did not return a figure.")
            
            print("✅ Plotting complete.")

        except Exception as e:
            print(f"❌ Plotting failed: {e}")
            print("  This can happen if analysis results contain NaN/inf due to an unstable beam or extreme E,I values.")
            print("  Consider checking the beam stability and E, I values carefully.")
            print("  Beam results (if available from library):")
            # Add any methods here that indeterminatebeam might have to show results textually
            # e.g. if hasattr(current_beam, 'get_reaction_results'): print(current_beam.get_reaction_results())
            traceback.print_exc()

# --- Button to Trigger Analysis ---
run_button = widgets.Button(description="▶️ Run Analysis & Plot", button_style='success', layout=widgets.Layout(width='250px'))
run_button.on_click(run_analysis_and_plot_action)

display(run_button, output_area)

Button(button_style='success', description='▶️ Run Analysis & Plot', layout=Layout(width='250px'), style=Butto…

Output()

In [4]:
import plotly # Import the main plotly library
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display as ipy_display # Use an alias

# Correct way to get Plotly version
print(f"Plotly version: {plotly.__version__}") 
print(f"Available Plotly renderers: {list(pio.renderers.keys())}")
print(f"Default Plotly renderer: {pio.renderers.default}")

# --- OPTIONALLY: Set a different default renderer globally here ---
# Try one of these if the default causes issues with the simple test below.
# If you change this, re-run this cell, then re-run the beam analysis cell.
# pio.renderers.default = 'jupyterlab'  # If in JupyterLab
# pio.renderers.default = 'notebook'    # If in Classic Jupyter Notebook & extensions are installed
# pio.renderers.default = 'vscode'      # If in VS Code
# pio.renderers.default = 'colab'       # If in Google Colab
# pio.renderers.default = 'iframe'      # Often a good fallback, self-contained
# print(f"Attempted to set Plotly default renderer to: {pio.renderers.default}")
# ---

try:
    print("\nCreating a simple Plotly scatter plot...")
    simple_fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])
    simple_fig.update_layout(title_text="Simple Plotly Test")

    print("Attempting to show simple Plotly figure with simple_fig.show()...")
    simple_fig.show() # Key test for .show()
    print("Simple Plotly figure .show() call completed (check if visible above).")

    # Alternative display method test
    # print("\nAttempting to show simple Plotly figure with IPython.display.display()...")
    # ipy_display(simple_fig)
    # print("Simple Plotly figure display() call completed (check if visible above).")

except Exception as e:
    print(f"❌ Error during simple Plotly test: {e}")
    import traceback
    traceback.print_exc()

Plotly version: 6.0.1


: 